In [8]:
import pandas as pd

df = pd.read_excel('/Users/simo/Data/btc/data1.xlsx', sheet_name='Order History 现货订单记录')

In [9]:
df.head()

,User ID 用户ID,Market ID 交易对,Price 价格,Qty 数量,Average Price 平均价,Remain Qty 剩余数量,Trade Qty 交易数量,Time 时间,Update Time 更新时间,Side 买卖交易方向,Status 交易状态,Type 挂单类型,Stop Price 止损价,Price Unit 计价单位,Amount Unit 数量单位
0,56164378,BTCUSDT,0.00000,1.70149,29257.450000,0.0000,1.70149,2023-08-16 02:20:55,2023-08-16 02:20:55,SELL,FILLED,MARKET,0.0,USDT,BTC
1,56164378,ETHUSDT,200.00000,0.10450,1913.000000,0.0000,0.10450,2023-07-17 23:40:34,2023-07-17 23:40:34,BUY,FILLED,MARKET,0.0,USDT,ETH
2,56164378,ETHUSDT,1912.82000,0.10450,0.000000,0.1045,0.00000,2023-07-17 23:40:12,2023-07-17 23:40:22,BUY,CANCELED,LIMIT,0.0,USDT,ETH
3,56164378,TRXUSDT,0.07975,1355.70000,0.079750,0.0000,1355.70000,2023-07-14 23:49:22,2023-07-14 23:49:22,BUY,FILLED,LIMIT,0.0,USDT,TRX
4,56164378,BTCUSDT,0.00000,10.80271,30398.797919,0.0000,10.80271,2023-06-27 03:24:40,2023-06-27 03:24:40,SELL,FILLED,MARKET,0.0,USDT,BTC


In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 22 18:51:55 2023

@author: zhouh
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

############################################################################################################################################################## 
# 读取Excel文件
file_path = r'/Users/simo/Data/btc/data1.xlsx'
sheet_name = "Order History 现货订单记录"
df = pd.read_excel(file_path, sheet_name)

# 优化列名
optimized_columns = {
    "User ID 用户ID": "User_ID",
    "Market ID 交易对": "Market_ID",
    "Price 价格": "Price",
    "Qty 数量": "Quantity",
    "Average Price 平均价": "Average_Price",
    "Remain Qty 剩余数量": "Remain_Quantity",
    "Trade Qty 交易数量": "Trade_Quantity",
    "Time 时间": "Time",
    "Update Time 更新时间": "Update_Time",
    "Side 买卖交易方向": "Buy_Sell",
    "Status 交易状态": "Trade_Status",
    "Type 挂单类型": "Order_Type",
    "Stop Price 止损价": "Stop_Price",
    "Price Unit 计价单位": "Price_Unit",
    "Amount Unit 数量单位": "Amount_Unit"
}

df.rename(columns=optimized_columns, inplace=True)

df.set_index("Time", inplace=True) # 将 "Time" 列设置为索引
df.sort_index(inplace=True) # 升序排序
df.index = pd.to_datetime(df.index)  # 将索引转换为 datetime 类型
df.index = df.index.strftime('%Y-%m-%d %H:%M:%S') # 格式化索引为 "YYYY-MM-DD HH:MM:SS"

total_trade_count = len(df) #取一下原始总的交易数量

df = df[df["Trade_Status"] == "FILLED"] #筛选出 "TradeStatus" 为 "FILLED" 的数据
total_trade_filled_count = len(df) #取一下实际完成的交易数量

# 保留所需的列
selected_columns = ["User_ID","Market_ID", "Price", "Quantity", "Average_Price", "Trade_Quantity", "Buy_Sell", "Price_Unit", "Amount_Unit"]
df = df[selected_columns]


############################################################################################################################################################## 
stable_coin = ['BUSD','USDT','USDC'] #这三个元素是当前biance上的稳定币代码
df.loc[:, 'stable_trade'] = df.apply(lambda row: row['Price_Unit'] in stable_coin and row['Amount_Unit'] in stable_coin, axis=1) #标识出买卖均为稳定币的交易（空转交易）
stable_trade_count = df['stable_trade'].sum() #取一下纯稳定币空转交易的数量

############################################################################################################################################################## 

# Time_diff列为两次filled交易之间的时间间隔
df['Time_diff'] = pd.to_datetime(df.index).to_series().diff()
df['Time_diff'] = df['Time_diff'].apply(lambda x: x.total_seconds()).fillna(0)

df_true = df[df['stable_trade']] #只取纯稳定币交易

df_true_pre = df[df['stable_trade']].shift(1) 
merged_df = pd.merge(df_true, df_true_pre, left_index=True, right_index=True, suffixes=('_now', '_pre')) #取纯稳定币交易之前的一次纯稳定币交易，并列进行分析

merged_df['Trade_Quantity_diff'] = merged_df['Trade_Quantity_now'] / merged_df['Trade_Quantity_pre'] - 1 # 匹配后一次交易相对上一次的数量差异，用于后续筛选

coin_hold = np.where(merged_df['Buy_Sell_pre'] == 'BUY', merged_df['Amount_Unit_pre'], merged_df['Price_Unit_pre']) # 前一次交易后手上持有的币种，用于和now这个后续交易使用币种关联
coin_used = np.where(merged_df['Buy_Sell_now'] == 'BUY', merged_df['Price_Unit_now'], merged_df['Amount_Unit_now']) # 当前交易使用的币种

merged_df['same_coin'] = coin_used == coin_hold

# 此版本的风险交易标识flag为TRUE的设置标准为：
# 1、当前交易花出去的币种等于前一次交易获得的币种，即coin_hold==coin_used；
# 2、Trade_Quantity_diff的绝对值小于0.01；
merged_df['same_coin'] = (coin_used == coin_hold)
merged_df['same_quant'] = merged_df['Trade_Quantity_diff'].abs() < 0.01
merged_df['risk_flag'] = (merged_df['same_coin'] & merged_df['same_quant'])

merged_df['trade_group'] = (~merged_df['risk_flag']).cumsum() #将连续风险交易视为一组交易从而对交易进行分组
merged_df['group_count'] = merged_df.groupby('trade_group')['trade_group'].transform('count')
# print(merged_df)
risk_trade_count = (merged_df['group_count'] > 1).sum() #取一下被标为risk flag的交易数量
output_sentence = """该用户在此区间共申报了{total_trade_count}笔交易，其中完成FILLED的交易共有{total_trade_filled_count}笔；属于稳定币之间的转换交易(无投资意义的转换操作)有{stable_trade_count}笔，而通过进一步分析认定为极大可能为风险交易的有{risk_trade_count}笔。""".format(
                    total_trade_count=total_trade_count,total_trade_filled_count=total_trade_filled_count,stable_trade_count=stable_trade_count,risk_trade_count=risk_trade_count)
print(output_sentence)

# 筛选出被标记为风险交易的记录
risk_trades = merged_df[merged_df['risk_flag']]

# 保存风险交易记录到新的DataFrame
risk_trades_df = risk_trades[["User_ID_now",'Market_ID_now', 'Price_now', 'Quantity_now', 'Average_Price_now', 'Trade_Quantity_now', 'Buy_Sell_now', 'Price_Unit_now', 'Amount_Unit_now', 'Time_diff_now', 'Trade_Quantity_diff', 'trade_group', 'group_count']]

# 如果需要，可以将这些记录导出到Excel文件
# risk_trades_df.to_excel('/path/to/save/risk_trades.xlsx')

# 显示部分风险交易记录
print(risk_trades_df.head())


该用户在此区间共申报了221笔交易，其中完成FILLED的交易共有159笔；属于稳定币之间的转换交易(无投资意义的转换操作)有136笔，而通过进一步分析认定为极大可能为风险交易的有106笔。
                     User_ID_now Market_ID_now  Price_now  Quantity_now  \
Time                                                                      
2022-01-05 20:09:51     56164378      BUSDUSDT     1.0001      39980.00   
2022-01-25 16:38:41     56164378      USDCBUSD     1.0001     106160.38   
2022-01-29 12:14:06     56164378      BUSDUSDT     0.9997       6125.00   
2022-02-03 16:08:34     56164378      USDCBUSD     1.0001     100231.36   
2022-02-05 00:11:54     56164378      USDCBUSD     0.9999     100000.00   

                     Average_Price_now  Trade_Quantity_now Buy_Sell_now  \
Time                                                                      
2022-01-05 20:09:51             1.0001            39980.00         SELL   
2022-01-25 16:38:41             1.0001           106160.38          BUY   
2022-01-29 12:14:06             0.9997             6125.00         SELL   
202

In [ ]:


# 对风险标识交易的交易间隔作图
# 筛选 risk_flag 为 True 的数据
true_data = merged_df[merged_df['risk_flag']]

# 将 Time_diff 列的数据除以 3600
plot_data = true_data['Time_diff_now'] / 3600

# 画样本分布图
plt.hist(plot_data, bins=50, color='blue', edgecolor='black')
plt.title('Sample Distribution of Time_diff for risk_flag=True')
plt.xlabel('Time_diff (hours)')
plt.ylabel('Frequency')
plt.show()

In [2]:
from binance_historical_data import BinanceDataDumper

data_dumper = BinanceDataDumper(
    path_dir_where_to_dump=".",
    asset_class="spot",  # spot, um, cm
    data_type="klines",  # aggTrades, klines, trades
    data_frequency="1m",
)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

####################################################### 数据加载预处理 ######################################################################### 


def btc_model(file_path):
    # DataSheet 1:
    # Order History 现货订单记录数据加载预处理
    sheet_name = "Order History 现货订单记录"
    df_order = pd.read_excel(file_path, sheet_name)

    # 优化列名
    optimized_columns = {
        "User ID 用户ID": "User_ID",
        "Market ID 交易对": "Market_ID",
        "Price 价格": "Price",
        "Qty 数量": "Quantity",
        "Average Price 平均价": "Average_Price",
        "Remain Qty 剩余数量": "Remain_Quantity",
        "Trade Qty 交易数量": "Trade_Quantity",
        "Time 时间": "Time",
        "Update Time 更新时间": "Update_Time",
        "Side 买卖交易方向": "Buy_Sell",
        "Status 交易状态": "Status",
        "Type 挂单类型": "Order_Type",
        "Stop Price 止损价": "Stop_Price",
        "Price Unit 计价单位": "Price_Unit",
        "Amount Unit 数量单位": "Amount_Unit"
    }

    selected_columns = ["Time", "Quantity", "Average_Price", "Trade_Quantity", "Buy_Sell", "Price_Unit", "Amount_Unit","Status"]
    df_order.rename(columns=optimized_columns, inplace=True)
    df_order = df_order[selected_columns] # 保留所需的列
    del sheet_name, optimized_columns, selected_columns

    # 新增 Currency_Out/Currency_Out_Amount 列
    df_order["Currency_Out"] = df_order.apply(
        lambda row: row["Price_Unit"] if row["Buy_Sell"] == "BUY" else row["Amount_Unit"],
        axis=1
    )

    df_order["Currency_Out_Amount"] = df_order.apply(
        lambda row: row["Average_Price"]*row["Trade_Quantity"] if row["Buy_Sell"] == "BUY" else row["Trade_Quantity"],
        axis=1
    )

    # 新增 Currency_In/Currency_In_Amount 列
    df_order["Currency_In"] = df_order.apply(
        lambda row: row["Amount_Unit"] if row["Buy_Sell"] == "BUY" else row["Price_Unit"],
        axis=1
    )

    df_order["Currency_In_Amount"] = df_order.apply(
        lambda row: row["Trade_Quantity"] if row["Buy_Sell"] == "BUY" else row["Average_Price"]*row["Trade_Quantity"],
        axis=1
    )

    df_order["Trade_Type"] = "order" 

    # DataSheet 2:
    # Deposit History 充值记录数据加载预处理
    sheet_name = "Deposit History 充值记录"
    df_deposit = pd.read_excel(file_path, sheet_name)

    # 优化列名
    optimized_columns = {
        "User ID 用户ID": "User_ID",
        "Currency 币种": "Currency_In",
        "Amount 数额": "Currency_In_Amount",
        "Account Type 帐户类型": "Account_Type",
        "BUSD 对应数额": "BUSD_Amount",
        "Deposit Address 接收地址": "Deposit_Address",
        "Source Address 发送地址": "Source_Address",
        "TXID 交易哈希": "TXID",
        "Create Time 时间": "Time",
        "Status 状态": "Status",
        "Network 网络": "Network",
        "CounterParty ID 内部划转方ID": "CounterParty_ID"
    }

    selected_columns = ["Time", "Currency_In", "Currency_In_Amount", "BUSD_Amount", "Status"]

    df_deposit.rename(columns=optimized_columns, inplace=True)
    df_deposit = df_deposit[selected_columns] # 保留所需的列
    del sheet_name, optimized_columns, selected_columns

    # 新增 Currency_Out/Currency_Out_Amount 列
    df_deposit["Currency_Out"] = ""
    df_deposit["Currency_Out_Amount"] = 0.0

    df_deposit["Trade_Type"] = "deposit" 

    # DataSheet 3:
    # Withdrawal History 提现记录数据加载预处理
    sheet_name = "Withdrawal History 提现记录"
    df_withdrawal = pd.read_excel(file_path, sheet_name)

    # 优化列名
    optimized_columns = {
        "User ID 用户ID": "User_ID",
        "Currency 币种": "Currency_Out",
        "Amount 数额": "Currency_Out_Amount",
        "Account Type 帐户类型": "Account_Type",
        "BUSD 对应数额": "BUSD_Amount",
        "Destination Address 目标地址": "Destination_Address",
        "Label/Tag/Memo 标签/备忘录": "Label_Tag_Memo",
        "txId 交易哈希": "TXID",
        "Apply Time 申请时间": "Time",
        "Status 状态": "Status",
        "Network 网络": "Network",
        "CounterParty ID 内部划转方ID": "CounterParty_ID"
    }


    selected_columns = ["Time", "Currency_Out", "Currency_Out_Amount", "BUSD_Amount", "Status"]

    df_withdrawal.rename(columns=optimized_columns, inplace=True)
    df_withdrawal = df_withdrawal[selected_columns] # 保留所需的列
    del sheet_name, optimized_columns, selected_columns

    # 新增 Currency_In/Currency_In_Amount 列
    df_withdrawal["Currency_In"] = ""
    df_withdrawal["Currency_In_Amount"] = 0.0

    df_withdrawal["Trade_Type"] = "withdrawal" 

    # DataSheet 4:
    # P2P 法币交易数据加载预处理
    sheet_name = "P2P 法币交易"
    df_RMB = pd.read_excel(file_path, sheet_name)

    # 优化列名
    optimized_columns = {
        "Order ID 订单ID": "Order_ID",
        "Ad number ad号": "Ad_Number",
        "Buy or Sell 买卖": "Buy_Sell",
        "Crypto 币种": "Crypto",
        "Amount 数额": "Amount",
        "Fiat Currency 法币": "Fiat_Currency",
        "Total Amount 总数额": "Total_Amount",
        "Unit Price 单价": "Unit_Price",
        "Take ID 对手方用户ID": "Take_ID",
        "Ad publisher ID 广告发行人": "Ad_Publisher_ID",
        "Create Time 创建时间": "Time",
        "Status 状态": "Status",
        "Payment method 支付方式": "Payment_Method",
        "Client 客户端": "Client",
        "Payment Time 支付时间": "Payment_Time",
        "Release time 放币时间": "Release_Time",
        "Update time 更新时间": "Update_Time"
    }


    selected_columns = ["Time", "Buy_Sell", "Crypto", "Amount", "Total_Amount", "Unit_Price", "Status"]

    df_RMB.rename(columns=optimized_columns, inplace=True)
    df_RMB = df_RMB[selected_columns] # 保留所需的列
    del sheet_name, optimized_columns, selected_columns

    # 新增 Currency_Out/Currency_Out_Amount 列
    df_RMB["Currency_Out"] = df_RMB.apply(
        lambda row: "" if row["Buy_Sell"] == "buy" else row["Crypto"],
        axis=1
    )

    df_RMB["Currency_Out_Amount"] = df_RMB.apply(
        lambda row: 0.0 if row["Buy_Sell"] == "buy" else row["Amount"],
        axis=1
    )

    # 新增 Currency_In/Currency_In_Amount 列
    df_RMB["Currency_In"] = df_RMB.apply(
        lambda row: row["Crypto"] if row["Buy_Sell"] == "buy" else "",
        axis=1
    )

    df_RMB["Currency_In_Amount"] = df_RMB.apply(
        lambda row: row["Amount"] if row["Buy_Sell"] == "buy" else 0.0,
        axis=1
    )

    df_RMB["Trade_Type"] = "RMB" 

    # DataSheet 5:
    # Binance Pay 币安支付数据加载预处理
    sheet_name = "Binance Pay 币安支付"
    df_binance = pd.read_excel(file_path, sheet_name)

    # 优化列名
    optimized_columns = {
        "Currency 币种": "Currency",
        "Amount 数额": "Amount",
        "Transaction Time 交易时间": "Time"
    }


    selected_columns = ["Time", "Currency", "Amount"]

    df_binance.rename(columns=optimized_columns, inplace=True)
    df_binance = df_binance[selected_columns] # 保留所需的列
    del sheet_name, optimized_columns, selected_columns

    # 新增 Currency_Out/Currency_Out_Amount 列
    df_binance["Currency_Out"] = df_binance.apply(
        lambda row: "" if row["Amount"] > 0 else row["Currency"],
        axis=1
    )

    df_binance["Currency_Out_Amount"] = df_binance.apply(
        lambda row: 0.0 if row["Amount"] > 0 else -row["Amount"],
        axis=1
    )

    # 新增 Currency_In/Currency_In_Amount 列
    df_binance["Currency_In"] = df_binance.apply(
        lambda row: row["Currency"] if row["Amount"] > 0 else "",
        axis=1
    )

    df_binance["Currency_In_Amount"] = df_binance.apply(
        lambda row: row["Amount"] if row["Amount"] > 0 else 0.0,
        axis=1
    )

    df_binance["Trade_Type"] = "binance" 


    ##################### 将5张数据表合并成一个统一格式的流入/流出 In/Out表格
    same_colums = ["Time", "Currency_In", "Currency_In_Amount", "Currency_Out", "Currency_Out_Amount", "Trade_Type"]



    merged_df = pd.concat([df_order[df_order["Status"] == "FILLED"][same_colums], 
                        df_deposit[df_deposit["Status"] == "成功"][same_colums],
                        df_withdrawal[df_withdrawal["Status"] == "Success"][same_colums],
                        df_RMB[df_RMB["Status"] == "Completed"][same_colums],
                        df_binance[same_colums]], ignore_index=True)



    merged_df = pd.concat([df_order[df_order["Status"] == "FILLED"][same_colums], 
                        df_deposit[df_deposit["Status"] == "成功"][same_colums],
                        df_withdrawal[df_withdrawal["Status"] == "Success"][same_colums],
                        df_RMB[df_RMB["Status"] == "Completed"][same_colums]], ignore_index=True)
    return merged_df

In [7]:
file_path = r'/Users/simo/Data/btc/data2.xlsx'
merged_df = btc_model(file_path=file_path)
merged_df.to_csv('./res2.csv')
print(merged_df)

ValueError: Worksheet named 'Binance Pay 币安支付' not found